##Download dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/GRAD_project/prepared_data/sample_data.csv")

In [3]:
print(data.keys())


Index(['Unnamed: 0', 'paper', 'slide'], dtype='object')


In [4]:
#remove extra column
data=data.drop(['Unnamed: 0'],axis=1)

In [5]:
print(data.keys())


Index(['paper', 'slide'], dtype='object')


In [ ]:
data['paper'] = data['paper'].astype('string')
data['slide'] = data['slide'].astype('string')

In [ ]:
print(data.head())

                                               paper                                              slide
0   1  Abstract Voice over IP (VoIP) systems are ...  Slide 1/18DCSL: Dependable Computing Systems L...
1  TOPHAT:Topology-basedHost-LevelAttributionforM...  10/26/20171TOPHAT: Topology based Host‐Level A...
2    1SLAM: Sleep-Wake Aware Local Monitoring in ...  Slide 1/27DCSL: Dependable Computing Systems L...
3  PRIVATUS:Wallet-FriendlyPrivacyProtectionforSm...  Slide 1/29  PRIVATUS: Wallet-Friendly Privacy ...
4  Snowpack:EfficientParameterChoiceforGPUKernels...  11/23/20171This work was performed under the a...


In [ ]:
print(len(data['paper'][0].split(" ")))

7353


In [ ]:
s=data.values.tolist()

In [ ]:
print(s)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Installation of transformers and checking the GPU 

In [ ]:
pip install Sentencepiece

     |████████████████████████████████| 1.2MB 9.6MB/s 


In [ ]:
!pip install --quiet transformers

!nvidia-smi

     |████████████████████████████████| 2.5MB 7.3MB/s 
     |████████████████████████████████| 3.3MB 35.7MB/s 
     |████████████████████████████████| 901kB 54.1MB/s 
Mon Jun 28 08:58:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+

In [ ]:
! pip install datasets  rouge-score nltk

     |████████████████████████████████| 245kB 8.3MB/s 
     |████████████████████████████████| 122kB 14.2MB/s 
     |████████████████████████████████| 245kB 14.3MB/s 


## Necessary Imports and model download

In [6]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(42)

## Initialize the mode and its tokenizer

In [7]:
#  tokenizer = T5Tokenizer.from_pretrained('t5-base')
 #t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
# tokenizer.save_pretrained("/content/drive/MyDrive/GRAD_project/prepared_data")

Saving directory (/content/drive/MyDrive/GRAD_project/prepared_data/tokenizer/) should be a directory


In [ ]:
# t5_model.save_pretrained("/content/drive/MyDrive/GRAD_project/prepared_data")

In [8]:
model_checkpoint = "t5-small"

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
#tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/GRAD_project/prepared_data")

In [ ]:
#t5_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/GRAD_project/prepared_data")

# **Preprocessing the data**


In [10]:
tokenizer("Hello, this one sentence!").input_ids

[8774, 6, 48, 80, 7142, 55, 1]

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))


{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [11]:

prefix = "Presentme"


max_input_length = 1024
max_target_length = 128

def preprocess_function(r):
    inputs = prefix +r['paper'] 
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    tmp = prefix +r['slide'] 

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tmp, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    #print(model_inputs)
    return model_inputs



In [ ]:
preprocess_function(data)

{'input_ids': [[18795, 526, 209, 20114, 12347, 147, 3857, 41, 553, 32, 4629, 61, 1002, 33, 3, 11866, 16, 9897, 38, 8, 748, 21, 9153, 1222, 2249, 2117, 147, 3857, 5275, 5, 282, 8, 9897, 13, 27371, 1002, 5386, 6, 79, 33, 271, 1426, 15, 26, 12, 315, 4217, 13, 2155, 11733, 7, 128, 13, 84, 33, 806, 12, 224, 1002, 11, 128, 1130, 3, 9, 879, 3275, 5, 27371, 1002, 7663, 633, 126, 2428, 12, 9874, 11733, 3, 31636, 23, 106, 2149, 41, 4309, 134, 61, 6553, 5, 1485, 6, 175, 1002, 5936, 1317, 18870, 21, 580, 758, 41, 15, 5, 122, 5, 6, 180, 4629, 61, 11, 331, 1929, 41, 15, 5, 122, 5, 6, 391, 7150, 137, 5212, 6, 8, 1002, 33, 8308, 16, 1405, 11, 5936, 8308, 820, 6, 8379, 11, 3, 20042, 725, 5, 9879, 6, 8, 6032, 12, 224, 1002, 8438, 3, 9, 508, 853, 6, 45, 20, 7419, 13, 313, 12, 14425, 7712, 6032, 5, 4213, 6, 8, 1002, 33, 26481, 729, 15, 1162, 11, 3115, 365, 633, 315, 6601, 3303, 7, 5, 86, 48, 1040, 6, 62, 4230, 8, 408, 13, 46, 2155, 11733, 10664, 358, 7774, 12, 27371, 1002, 6, 718, 6508, 4309, 8087, 41, 20

{'input_ids': [[18795, 526, 209, 20114, 12347, 147, 3857, 41, 553, 32, 4629, 61, 1002, 33, 3, 11866, 16, 9897, 38, 8, 748, 21, 9153, 1222, 2249, 2117, 147, 3857, 5275, 5, 282, 8, 9897, 13, 27371, 1002, 5386, 6, 79, 33, 271, 1426, 15, 26, 12, 315, 4217, 13, 2155, 11733, 7, 128, 13, 84, 33, 806, 12, 224, 1002, 11, 128, 1130, 3, 9, 879, 3275, 5, 27371, 1002, 7663, 633, 126, 2428, 12, 9874, 11733, 3, 31636, 23, 106, 2149, 41, 4309, 134, 61, 6553, 5, 1485, 6, 175, 1002, 5936, 1317, 18870, 21, 580, 758, 41, 15, 5, 122, 5, 6, 180, 4629, 61, 11, 331, 1929, 41, 15, 5, 122, 5, 6, 391, 7150, 137, 5212, 6, 8, 1002, 33, 8308, 16, 1405, 11, 5936, 8308, 820, 6, 8379, 11, 3, 20042, 725, 5, 9879, 6, 8, 6032, 12, 224, 1002, 8438, 3, 9, 508, 853, 6, 45, 20, 7419, 13, 313, 12, 14425, 7712, 6032, 5, 4213, 6, 8, 1002, 33, 26481, 729, 15, 1162, 11, 3115, 365, 633, 315, 6601, 3303, 7, 5, 86, 48, 1040, 6, 62, 4230, 8, 408, 13, 46, 2155, 11733, 10664, 358, 7774, 12, 27371, 1002, 6, 718, 6508, 4309, 8087, 41, 20

In [12]:
ready_data =[preprocess_function(r) for i, r in data.iterrows()]


In [13]:
ready_data[0]

{'input_ids': [18795, 526, 209, 20114, 12347, 147, 3857, 41, 553, 32, 4629, 61, 1002, 33, 3, 11866, 16, 9897, 38, 8, 748, 21, 9153, 1222, 2249, 2117, 147, 3857, 5275, 5, 282, 8, 9897, 13, 27371, 1002, 5386, 6, 79, 33, 271, 1426, 15, 26, 12, 315, 4217, 13, 2155, 11733, 7, 128, 13, 84, 33, 806, 12, 224, 1002, 11, 128, 1130, 3, 9, 879, 3275, 5, 27371, 1002, 7663, 633, 126, 2428, 12, 9874, 11733, 3, 31636, 23, 106, 2149, 41, 4309, 134, 61, 6553, 5, 1485, 6, 175, 1002, 5936, 1317, 18870, 21, 580, 758, 41, 15, 5, 122, 5, 6, 180, 4629, 61, 11, 331, 1929, 41, 15, 5, 122, 5, 6, 391, 7150, 137, 5212, 6, 8, 1002, 33, 8308, 16, 1405, 11, 5936, 8308, 820, 6, 8379, 11, 3, 20042, 725, 5, 9879, 6, 8, 6032, 12, 224, 1002, 8438, 3, 9, 508, 853, 6, 45, 20, 7419, 13, 313, 12, 14425, 7712, 6032, 5, 4213, 6, 8, 1002, 33, 26481, 729, 15, 1162, 11, 3115, 365, 633, 315, 6601, 3303, 7, 5, 86, 48, 1040, 6, 62, 4230, 8, 408, 13, 46, 2155, 11733, 10664, 358, 7774, 12, 27371, 1002, 6, 718, 6508, 4309, 8087, 41, 207

In [ ]:
# tokenized_datasets = d1.map(preprocess_function, batched=True)

AttributeError: ignored

## Fine-tuning the model

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [16]:


batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)



In [17]:
from datasets import load_metric

metric = load_metric("rouge")


In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [21]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ready_data,
    # eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [22]:
trainer.train()

***** Running training *****
  Num examples = 79
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5


RuntimeError: ignored

## Initialize the optimizer

In [ ]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model2.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model2.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [ ]:
print(f"GPU is available ?:{torch.cuda.is_available()}")
device=torch.device("cuda:0")
t5_model2.to(device)

GPU is available ?:True


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
ready_data =[]
for i,r in data.iterrows():
    input=r['paper']
    output=r['slide']
    Paper = "summary: "+input+ " </s>"
    slide = output+" </s>"
     
    tokenized_inp = tokenizer.encode_plus(Paper,  max_length=100, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(slide, max_length=100, pad_to_max_length=True,return_tensors="pt")
     
    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]
    
    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]
  
    ready_data.append([np.array(input_ids , dtype='int' ) ,np.array(attention_mask , dtype='int' ), np.array(lm_labels, dtype='int' ) ,np.array(decoder_attention_mask, dtype='int' )])

In [ ]:
# ready_data =[]
# for i,r in data.iterrows():
#     input=r['paper']
#     output=r['slide']
#     Paper = "summary: "+input+ " </s>"
#     slide = output+" </s>"
     
#     tokenized_inp = tokenizer2.encode_plus(Paper,  max_length=100, pad_to_max_length=True,return_tensors="pt")
#     tokenized_output = tokenizer2.encode_plus(slide, max_length=100, pad_to_max_length=True,return_tensors="pt")
     
#     input_ids  = tokenized_inp["input_ids"]
#     attention_mask = tokenized_inp["attention_mask"]
    
#     lm_labels= tokenized_output["input_ids"]
#     decoder_attention_mask=  tokenized_output["attention_mask"]
  
#     ready_data.append([np.array(input_ids , dtype='int' ), np.array(lm_labels, dtype='int' ) ])

In [ ]:
print(len(ready_data))
print(len(ready_data[0]))
ready0= np.array(ready_data )
print(ready0.shape)
ready = torch.FloatTensor(np.stack(ready0))
print(f"data shape is :{ready.shape}")
ready=ready.long()
ready.to(device)

In [ ]:
# dataloader 

# Pla pla

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("xsum")


Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


In [120]:
raw_datasets2= load_dataset("imdb")

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


In [123]:
raw_datasets2

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function1, batched=True)

In [ ]:
tokenized_data1 = map(data1,preprocess_function1)

TypeError: ignored

In [95]:
data1=data.to_dict('records')

In [ ]:
data1

In [106]:
ready_data =[preprocess_function(r) for r in data1]


In [107]:
ready_data[0]

{'input_ids': [18795, 526, 209, 20114, 12347, 147, 3857, 41, 553, 32, 4629, 61, 1002, 33, 3, 11866, 16, 9897, 38, 8, 748, 21, 9153, 1222, 2249, 2117, 147, 3857, 5275, 5, 282, 8, 9897, 13, 27371, 1002, 5386, 6, 79, 33, 271, 1426, 15, 26, 12, 315, 4217, 13, 2155, 11733, 7, 128, 13, 84, 33, 806, 12, 224, 1002, 11, 128, 1130, 3, 9, 879, 3275, 5, 27371, 1002, 7663, 633, 126, 2428, 12, 9874, 11733, 3, 31636, 23, 106, 2149, 41, 4309, 134, 61, 6553, 5, 1485, 6, 175, 1002, 5936, 1317, 18870, 21, 580, 758, 41, 15, 5, 122, 5, 6, 180, 4629, 61, 11, 331, 1929, 41, 15, 5, 122, 5, 6, 391, 7150, 137, 5212, 6, 8, 1002, 33, 8308, 16, 1405, 11, 5936, 8308, 820, 6, 8379, 11, 3, 20042, 725, 5, 9879, 6, 8, 6032, 12, 224, 1002, 8438, 3, 9, 508, 853, 6, 45, 20, 7419, 13, 313, 12, 14425, 7712, 6032, 5, 4213, 6, 8, 1002, 33, 26481, 729, 15, 1162, 11, 3115, 365, 633, 315, 6601, 3303, 7, 5, 86, 48, 1040, 6, 62, 4230, 8, 408, 13, 46, 2155, 11733, 10664, 358, 7774, 12, 27371, 1002, 6, 718, 6508, 4309, 8087, 41, 207

In [102]:
print(raw_datasets["train"][0])

{'document': 'Recent reports have linked some France-based players with returns to Wales.\n"I\'ve always felt - and this is with my rugby hat on now; this is not region or WRU - I\'d rather spend that money on keeping players in Wales," said Davies.\nThe WRU provides £2m to the fund and £1.3m comes from the regions.\nFormer Wales and British and Irish Lions fly-half Davies became WRU chairman on Tuesday 21 October, succeeding deposed David Pickering following governing body elections.\nHe is now serving a notice period to leave his role as Newport Gwent Dragons chief executive after being voted on to the WRU board in September.\nDavies was among the leading figures among Dragons, Ospreys, Scarlets and Cardiff Blues officials who were embroiled in a protracted dispute with the WRU that ended in a £60m deal in August this year.\nIn the wake of that deal being done, Davies said the £3.3m should be spent on ensuring current Wales-based stars remain there.\nIn recent weeks, Racing Metro fla

In [119]:
print(tokenized_datasets["train"])

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
    num_rows: 204045
})


## Trainer API

In [108]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [135]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=ready_data
)


In [136]:
trainer.train()

***** Running training *****
  Num examples = 79
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30


RuntimeError: ignored

## Training the mode

In [1]:
t5_model2.train()

epochs = 1

for epoch in range(epochs):
    for dataitem in ready :
        input_ids = dataitem[0][0]
        attention_mask= dataitem[1][0]
        lm_labels= dataitem[2][0]
        decoder_attention_mask= dataitem[3][0]
        input_ids.to(device)
        attention_mask.to(device)
        decoder_attention_mask.to(device)
        lm_labels.to(device)
        
    # the forward function automatically creates the correct decoder_input_ids
    expected_output = t5_model(input_ids=input_ids, labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask ).to(device)
    loss = expected_output[0]

    loss.backward()
    optimizer.step()

NameError: ignored

## Testing the model

In [ ]:

test_sent = data['paper'][0]+ " </s>"
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=100,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)



NameError: ignored